<a href="https://colab.research.google.com/github/NapoliD/telecom/blob/master/Clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Ejemplo si el archivo esta cargado en el entorno actual
os.chdir('/content/')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [5]:
datos = pd.read_csv('cliente.csv', sep= ',')
datos

,Plan_mes,mes,nro_cliente
0,95729,1,1
1,96410,1,3
2,96400,1,2
3,95729,1,5
4,95420,1,4
...,...,...,...
32613,96806,3,10897
32614,96890,3,10895
32615,96806,3,10898
32616,96806,3,10900


In [ ]:
datos

In [6]:
datos.groupby('Plan_mes')['nro_cliente'].count().sort_values(ascending=False)

Plan_mes
95982    2857
96385    2509
96133    1438
97017    1342
96890    1329
         ... 
94419       1
94424       1
95348       1
94900       1
91358       1
Name: nro_cliente, Length: 196, dtype: int64

In [7]:
clientes=datos.groupby('nro_cliente')['Plan_mes'].aggregate(lambda x: x.tolist()).reset_index()
clientes

,nro_cliente,Plan_mes
0,1,"[95729, 95729, 96810]"
1,2,"[96400, 96400, 96400]"
2,3,"[96410, 96410, 96410]"
3,4,"[95420, 96413, 96413]"
4,5,"[95729, 96810, 96810]"
...,...,...
10895,10896,"[96806, 96806]"
10896,10897,"[96806, 96806]"
10897,10898,"[96806, 96806]"
10898,10899,"[96766, 96770]"


In [8]:
planes=clientes['Plan_mes']

In [9]:
planes=pd.DataFrame(planes).astype(str)

In [10]:
planes=pd.DataFrame(planes['Plan_mes'].apply(lambda x: pd.Series(x.split(','))))

In [11]:
planes

,0,1,2
0,[95729,95729,96810]
1,[96400,96400,96400]
2,[96410,96410,96410]
3,[95420,96413,96413]
4,[95729,96810,96810]
...,...,...,...
10895,[96806,96806],NaN
10896,[96806,96806],NaN
10897,[96806,96806],NaN
10898,[96766,96770],NaN


In [ ]:
mes1=pd.DataFrame(planes[0].map(lambda x: x.lstrip('[')).astype(int))
mes1.columns=['mes1']
mes1

In [ ]:
mes2=planes[1].astype(str).apply(lambda x: pd.Series(x.split(']')))
mes2=pd.DataFrame(mes2[0].astype(int))
mes2.columns=['mes2']
mes2

In [22]:
mes3=planes[2].astype(str).apply(lambda x: pd.Series(x.split(']')))
mes3=pd.DataFrame(mes3[0].replace('nan',0).astype(int))
mes3.columns=['mes3']
mes3

,mes3
0,96810
1,96400
2,96410
3,96413
4,96810
...,...
10895,0
10896,0
10897,0
10898,0


In [59]:
base_final=pd.concat([clientes.reset_index(drop=True),
                      mes1.reset_index(drop=True),mes2.reset_index(drop=True),mes3.reset_index(drop=True)], axis=1)

In [60]:
base_final

,nro_cliente,Plan_mes,mes1,mes2,mes3
0,1,"[95729, 95729, 96810]",95729,95729,96810
1,2,"[96400, 96400, 96400]",96400,96400,96400
2,3,"[96410, 96410, 96410]",96410,96410,96410
3,4,"[95420, 96413, 96413]",95420,96413,96413
4,5,"[95729, 96810, 96810]",95729,96810,96810
...,...,...,...,...,...
10895,10896,"[96806, 96806]",96806,96806,0
10896,10897,"[96806, 96806]",96806,96806,0
10897,10898,"[96806, 96806]",96806,96806,0
10898,10899,"[96766, 96770]",96766,96770,0


In [ ]:
base_final.drop_duplicates(subset='nro_cliente',keep='first')

In [61]:
base_final['cambio_plan']=np.where( (base_final['mes1']!=base_final['mes2']),1,0)
base_final['cambio_plan1']=np.where((base_final['mes1']!=base_final['mes3']) & (base_final['mes3']!=0),1,0)
base_final['cambio_plan2']=np.where( (base_final['mes2']!=base_final['mes3']) &  (base_final['mes3']!=0),1,0)
base_final['cambio_plan_1_si_0_no']=np.where( (base_final['cambio_plan']==1) | (base_final['cambio_plan1']==1) | (base_final['cambio_plan2']==1),1,0)
base_final

,nro_cliente,Plan_mes,mes1,mes2,mes3,cambio_plan,cambio_plan1,cambio_plan2,cambio_plan_1_si_0_no
0,1,"[95729, 95729, 96810]",95729,95729,96810,0,1,1,1
1,2,"[96400, 96400, 96400]",96400,96400,96400,0,0,0,0
2,3,"[96410, 96410, 96410]",96410,96410,96410,0,0,0,0
3,4,"[95420, 96413, 96413]",95420,96413,96413,1,1,0,1
4,5,"[95729, 96810, 96810]",95729,96810,96810,1,1,0,1
...,...,...,...,...,...,...,...,...,...
10895,10896,"[96806, 96806]",96806,96806,0,0,0,0,0
10896,10897,"[96806, 96806]",96806,96806,0,0,0,0,0
10897,10898,"[96806, 96806]",96806,96806,0,0,0,0,0
10898,10899,"[96766, 96770]",96766,96770,0,1,0,0,1


In [62]:
base_final2=base_final.loc[:,['nro_cliente','cambio_plan_1_si_0_no']]
base_final_clientes=datos.merge(base_final2, on='nro_cliente', how='left')
base_final_clientes


,provincia,nro_cliente,cambio_plan_1_si_0_no
0,Chaco,1,1
1,Santa Fe,2,0
2,Córdoba,3,0
3,Salta,4,1
4,Santa Fe,5,1
...,...,...,...
10895,Catamarca,10896,0
10896,Buenos Aires,10897,0
10897,San Juan,10898,0
10898,Chaco,10899,1


In [49]:
base_final_clientes.to_csv("Clientes_cambio_plan.csv",index=False)

In [63]:
base_final2

,nro_cliente,cambio_plan_1_si_0_no
0,1,1
1,2,0
2,3,0
3,4,1
4,5,1
...,...,...
10895,10896,0
10896,10897,0
10897,10898,0
10898,10899,1


<h2> Cuantos clientes que se encuentran en CABA cambiaron de plan.</h2>

In [56]:
provincia = pd.read_csv('provincia.csv',sep=';',encoding='latin-1',decimal=',',thousands='.')
provincia

,provincia,nro_cliente
0,Chaco,1
1,Santa Fe,2
2,Córdoba,3
3,Salta,4
4,Santa Fe,5
...,...,...
10895,Catamarca,10896
10896,Buenos Aires,10897
10897,San Juan,10898
10898,Chaco,10899


In [64]:
provincia2=base_final2.merge(provincia, on='nro_cliente', how='left')
provincia2

,nro_cliente,cambio_plan_1_si_0_no,provincia
0,1,1,Chaco
1,2,0,Santa Fe
2,3,0,Córdoba
3,4,1,Salta
4,5,1,Santa Fe
...,...,...,...
10895,10896,0,Catamarca
10896,10897,0,Buenos Aires
10897,10898,0,San Juan
10898,10899,1,Chaco


In [78]:
cambiaron_caba=provincia2[(provincia2['cambio_plan_1_si_0_no']==1) & (provincia2['provincia'].str.contains("Ciudad Aut"))]
cant_camb=cambiaron_caba['nro_cliente'].count()
print(f'La cantidad de clientes que cambiaron de plan y encuentran en CABA son: {cant_camb} clientes')

La cantidad de clientes que cambiaron de plan y encuentran en CABA son: 706 clientes


Cant de clientes que se encuentran en CABA que cambiaron de plan: